# Santander Kaggle competition

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
import seaborn as sns
%matplotlib inline

In [16]:
train_data = pd.read_csv('train-2.csv')
test_data = pd.read_csv('test-2.csv')
sample_submission = pd.read_csv('sample_submission-3.csv')

In [20]:
id_test = test_data['ID']
y_train = train_data['TARGET'].values
X_train = train_data.drop(['ID','TARGET'], axis=1).values
X_test = test_data.drop(['ID'], axis=1).values
clf = RandomForestClassifier(n_estimators=100, max_depth=17, random_state=1)
scores = cross_validation.cross_val_score(clf, X_train, y_train, scoring='roc_auc', cv=5) 
print(scores.mean())
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)

0.821360825487


NameError: name 'pandas' is not defined

In [21]:
submission = pd.DataFrame({"ID":id_test, "TARGET":y_pred[:,1]})
submission.to_csv("submission.csv", index=False)